# 📥 Notebook 01 – Téléchargement des Spectres DR5

Ce notebook télécharge un lot de spectres .fits.gz du catalogue LAMOST DR5 à l’aide du script `dr5_downloader.py`.  
Il est conçu pour être exécuté **seulement lorsque de nouveaux spectres sont nécessaires**, afin d’alimenter le répertoire `data/raw/` en données brutes.

## ⚙️ Paramètres principaux

- `limit` : Nombre de plans de spectres à télécharger (chaque plan contient plusieurs spectres).
- `max_spectres` : Nombre maximum de spectres à télécharger.
- ``Workers`` : combien de ``.fits.gz`` en parallèle - Phase 2 du téléchargement.
- ``Scrape`` : combien de pages de plans analysées en parallèle - Phase 1 du téléchargement.

**Valeurs conseillées (règles rapides)**

- 1–3 plans : **Scrape** = 1–3
- 5–10 plans : **Scrape** = 6–8
- 10–30 plans : **Scrape** = 8–12
- Garde **Workers** (téléchargements) plus haut que Scrape en général, p.ex. **Workers 12–20, Scrape 6–12**.
<br/>

**Exécution** : Lance la cellule ci-dessous pour télécharger de nouveaux spectres depuis le catalogue public DR5.

In [1]:
from IPython.display import display, Markdown 
from utils import setup_project_env
from tools.spectra_manager import SpectraManager

# Initialisation de l'environnement et des chemins
paths = setup_project_env()

# Instanciation du gestionnaire de spectres
manager = SpectraManager(paths)

manager.interactive_downloader()

[INFO] Racine du projet détectée : C:\Users\alexb\Documents\Projets\AstroSpectro
[INFO] Dossier 'src' ajouté au sys.path.


### 1. Téléchargement des Spectres

Utilisez l’interface ci-dessous pour lancer un nouveau téléchargement. Les logs de progression s’afficheront en temps réel.

**Astuces** : Workers = téléchargements parallèles; Scrape = parallélisme d’analyse des pages plan; Par plan = coupe la liste par plan; Valider FITS = vérif header; Dry‑run = aperçu sans écriture.

Button(button_style='success', description='Lancer le téléchargement…', icon='download', style=ButtonStyle())

HTML(value='')

Output()

#

# 🧹 Option de Nettoyage
Il peut arriver que tu souhaites nettoyer entièrement le répertoire ``data/raw/`` avant de relancer un téléchargement, notamment pour repartir d’un dossier vierge ou libérer de l’espace.

Cette option de nettoyage intègre deux niveaux de protection pour éviter toute suppression accidentelle :

- Confirmation interactive :
Un bouton de confirmation via un widget (ou une autre méthode comme input()) demande explicitement ton accord avant d’exécuter la suppression.

- Sauvegarde automatique :
Avant de vider le répertoire, le script réalise une copie complète de data/raw/ dans un dossier temporaire nommé data/raw_backup/.

Cela te permet de restaurer tes spectres si jamais tu as validé la suppression par erreur.

**Important :**

- Le dossier de sauvegarde est écrasé à chaque nouvelle exécution de l’option.

- Tu peux restaurer des fichiers en récupérant les spectres depuis data/raw_backup/ tant que tu n’as pas relancé un nettoyage.

**Pourquoi utiliser cette option ?**

- Repartir sur un lot de spectres neuf sans mélange avec d’anciens fichiers.
- Tester un workflow complet à partir d’un état initial propre.
- Gagner en traçabilité et éviter les erreurs lors de sessions consécutives de téléchargement.

In [ ]:
from IPython.display import display, Markdown 

display(Markdown("--- \n### 2. Option de Nettoyage"))
display(Markdown("Si nécessaire, vous pouvez nettoyer le répertoire `data/raw/` pour repartir de zéro. Une sauvegarde sera automatiquement créée dans `data/raw_backup/`."))
manager.interactive_cleaner()

#